<a href="https://colab.research.google.com/github/saida-chalouach/incident_detection-llm-fine_tuning/blob/main/Fine_Tuning_Mistral_Incidents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Fine-tuning Mistral pour la Détection d'Incidents Web

Ce notebook vous guide à travers toutes les étapes du fine-tuning dans Google Colab.

## ⚙️ Configuration requise
- **GPU**: T4 (gratuit) ou A100 (Colab Pro)
- **Runtime**: GPU activé

## 📋 Étapes
1. Configuration de l'environnement
2. Upload et préparation des données
3. Fine-tuning du modèle
4. Évaluation
5. Test et utilisation

## 🔧 ÉTAPE 1: Configuration de l'environnement

In [ ]:
# Vérifier le GPU disponible
!nvidia-smi

In [1]:
# Installation des dépendances avec versions compatibles pour Colab
print("📦 Installation des dépendances...")

# D'abord, installer torch compatible avec l'environnement Colab
!pip install -q torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1

# Puis installer le reste des packages
!pip install -q -U transformers datasets accelerate peft bitsandbytes
!pip install pandas==2.2.2 numpy==2.1.0 --break-system-packages

print("✅ Installation terminée!")

📦 Installation des dépendances...
✅ Installation terminée!


In [2]:
# Imports
import torch
import pandas as pd
import json
import random
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from datasets import load_dataset

print("✅ Imports réussis")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🎮 CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")

✅ Imports réussis
🔥 PyTorch version: 2.5.1+cu124
🎮 CUDA disponible: True
🎯 GPU: Tesla T4


## 📤 ÉTAPE 2: Upload et préparation des données

**Instructions:**
1. Exécutez la cellule ci-dessous
2. Cliquez sur "Choose Files"
3. Sélectionnez votre fichier `data_stage_2A.txt`

In [3]:
from google.colab import files

print("📂 Upload de votre fichier de données...")
uploaded = files.upload()

# Récupérer le nom du fichier uploadé
data_file = list(uploaded.keys())[0]
print(f"\n✅ Fichier uploadé: {data_file}")

📂 Upload de votre fichier de données...


Saving data_stage_2A.txt to data_stage_2A.txt

✅ Fichier uploadé: data_stage_2A.txt


In [4]:
# Fonction de parsing des données
def parse_data_file(file_path):
    """Parse le fichier texte et extrait les données structurées"""
    data = []

    print(f"📂 Lecture du fichier: {file_path}")
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    for i in range(len(lines)):
        line = lines[i].strip()
        if line.startswith('A: '):
            values = line[3:].split(';')
            if len(values) >= 9:
                record = {
                    'heartbeat_id': values[0],
                    'is_important': values[1],
                    'monitor_name': values[2],
                    'monitor_id': values[3],
                    'heartbeat_status': values[4],
                    'heartbeat_msg': values[5],
                    'heartbeat_time': values[6],
                    'heartbeat_ping': values[7],
                    'duration': values[8]
                }
                data.append(record)

    print(f"✅ {len(data)} enregistrements extraits")
    return pd.DataFrame(data)

# Parser les données
df = parse_data_file(data_file)
print(f"\n📊 DataFrame créé: {df.shape[0]} lignes, {df.shape[1]} colonnes")
df.head()

📂 Lecture du fichier: data_stage_2A.txt
✅ 1666 enregistrements extraits

📊 DataFrame créé: 1666 lignes, 9 colonnes


,heartbeat_id,is_important,monitor_name,monitor_id,heartbeat_status,heartbeat_msg,heartbeat_time,heartbeat_ping,duration
0,41837805,0,Frontend PiTransfer Standalone,67,1,200 - OK,2024-07-05 02:37:28.238,49,60
1,41837806,0,Frontend PiDF Standalone,69,1,200 - OK,2024-07-05 02:37:28.307,40,60
2,41837807,0,Socket Chat,35,1,200 - OK,2024-07-05 02:37:28.426,43,60
3,41837808,0,New Media Server 9,59,1,200 - OK,2024-07-05 02:37:30.179,15,60
4,41837809,0,S3 Minio Store,63,1,200 - OK,2024-07-05 02:37:30.358,40,60


In [5]:
# Fonction de détection d'incidents
def detect_incident(row):
    """Détermine si un heartbeat indique un incident"""
    # Status 0 = Down
    if row['heartbeat_status'] == '0':
        return True, 'Service down - heartbeat status is 0'

    # Message vide ou erreur
    msg = row['heartbeat_msg'].strip()
    if not msg or msg == '':
        return True, 'No response message received'

    # Codes d'erreur HTTP
    error_codes = ['400', '401', '403', '404', '500', '502', '503', '504']
    if any(code in msg for code in error_codes):
        return True, f'HTTP error detected: {msg}'

    # Ping anormalement élevé (> 250ms)
    try:
        ping_str = row['heartbeat_ping'].strip()
        if ping_str:
            ping = float(ping_str)
            if ping > 250:
                return True, f'High latency detected: {ping}ms'
    except (ValueError, AttributeError):
        pass

    return False, 'Normal operation - all metrics within acceptable range'

# Appliquer la détection
print("🔍 Détection des incidents...")
df['is_incident'], df['incident_reason'] = zip(*df.apply(detect_incident, axis=1))

print(f"\n📊 Statistiques:")
print(f"  Total: {len(df)} enregistrements")
print(f"  Incidents: {df['is_incident'].sum()} ({df['is_incident'].sum()/len(df)*100:.1f}%)")
print(f"  Normaux: {(~df['is_incident']).sum()} ({(~df['is_incident']).sum()/len(df)*100:.1f}%)")

🔍 Détection des incidents...

📊 Statistiques:
  Total: 1666 enregistrements
  Incidents: 35 (2.1%)
  Normaux: 1631 (97.9%)


In [6]:
# Créer les échantillons d'entraînement
def create_training_samples(df, num_variations=2):
    """Crée des échantillons d'entraînement variés"""
    samples = []

    print(f"🔄 Génération des échantillons d'entraînement...")

    for idx, row in df.iterrows():
        monitor = row['monitor_name']
        status = row['heartbeat_status']
        msg = row['heartbeat_msg']
        ping = row['heartbeat_ping']
        time = row['heartbeat_time']
        is_incident = row['is_incident']
        reason = row['incident_reason']

        # Templates de questions
        question_templates = [
            f"Analyser le statut du service {monitor}: status={status}, message={msg}, ping={ping}ms",
            f"Le service {monitor} fonctionne-t-il correctement? Status: {status}, Response: {msg}",
            f"Vérifier l'état de {monitor}. Status={status}, Latency={ping}ms",
            f"Diagnostic pour {monitor}: {msg}, latence {ping}ms",
        ]

        # Réponses
        if is_incident:
            answer_templates = [
                f"🚨 INCIDENT DÉTECTÉ sur {monitor}. Raison: {reason}. Action requise immédiatement.",
                f"⚠️ Alerte: {monitor} rencontre un problème - {reason}. Intervention nécessaire.",
                f"Problème identifié sur {monitor}: {reason}. Statut critique.",
            ]
        else:
            answer_templates = [
                f"✅ {monitor} fonctionne normalement. Status: {status}, {msg}. Tous les indicateurs sont au vert.",
                f"Aucun incident détecté. {monitor} est opérationnel. Latence: {ping}ms.",
                f"Service {monitor} en bon état de fonctionnement. {msg}",
            ]

        # Générer variations
        for _ in range(num_variations):
            question = random.choice(question_templates)
            answer = random.choice(answer_templates)

            samples.append({
                'instruction': question,
                'output': answer,
                'input': '',
                'is_incident': is_incident,
            })

    print(f"✅ {len(samples)} échantillons générés")
    return samples

# Générer les échantillons
samples = create_training_samples(df, num_variations=2)

🔄 Génération des échantillons d'entraînement...
✅ 3332 échantillons générés


In [7]:
# Équilibrer et diviser le dataset
print("⚖️ Équilibrage du dataset...")

incident_samples = [s for s in samples if s['is_incident']]
normal_samples = [s for s in samples if not s['is_incident']]

print(f"  Incidents: {len(incident_samples)}")
print(f"  Normaux: {len(normal_samples)}")

# Équilibrer
min_count = min(len(incident_samples), len(normal_samples))
incident_samples = random.sample(incident_samples, min_count)
normal_samples = random.sample(normal_samples, min_count)

all_samples = incident_samples + normal_samples
random.shuffle(all_samples)

print(f"\n📊 Dataset équilibré: {len(all_samples)} échantillons")

# Split train/val/test (80/10/10)
train_data, temp_data = train_test_split(
    all_samples,
    test_size=0.2,
    random_state=42,
    stratify=[s['is_incident'] for s in all_samples]
)

val_data, test_data = train_test_split(
    temp_data,
    test_size=0.5,
    random_state=42,
    stratify=[s['is_incident'] for s in temp_data]
)

print(f"\n📈 Répartition:")
print(f"  Train: {len(train_data)} ({len(train_data)/len(all_samples)*100:.1f}%)")
print(f"  Validation: {len(val_data)} ({len(val_data)/len(all_samples)*100:.1f}%)")
print(f"  Test: {len(test_data)} ({len(test_data)/len(all_samples)*100:.1f}%)")

⚖️ Équilibrage du dataset...
  Incidents: 70
  Normaux: 3262

📊 Dataset équilibré: 140 échantillons

📈 Répartition:
  Train: 112 (80.0%)
  Validation: 14 (10.0%)
  Test: 14 (10.0%)


In [8]:
# Sauvegarder les datasets
Path('data').mkdir(exist_ok=True)

for split_name, split_data in [('train', train_data), ('val', val_data), ('test', test_data)]:
    with open(f'data/{split_name}_data.jsonl', 'w', encoding='utf-8') as f:
        for sample in split_data:
            f.write(json.dumps(sample, ensure_ascii=False) + '\n')
    print(f"✅ data/{split_name}_data.jsonl sauvegardé")

print("\n🎉 Préparation des données terminée!")

✅ data/train_data.jsonl sauvegardé
✅ data/val_data.jsonl sauvegardé
✅ data/test_data.jsonl sauvegardé

🎉 Préparation des données terminée!


## 🏋️ ÉTAPE 3: Fine-tuning du modèle

Cette étape peut prendre **2-4 heures** sur T4 gratuit, ou **30-60 minutes** sur A100.

In [9]:
# Configuration du modèle
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
OUTPUT_DIR = "./mistral-incident-detector"

print("="*60)
print("📦 Chargement de Mistral-7B...")
print("="*60)
print("⏰ Cela peut prendre 5-10 minutes...\n")

📦 Chargement de Mistral-7B...
⏰ Cela peut prendre 5-10 minutes...



In [10]:
# Installation de bitsandbytes pour la quantification 4-bit
print("📦 Installation de bitsandbytes...")

!pip install -q bitsandbytes>=0.46.1

print("✅ Installation terminée!")
print("\n⚠️ IMPORTANT: Redémarrez le runtime maintenant")
print("Runtime → Restart runtime")

📦 Installation de bitsandbytes...
✅ Installation terminée!

⚠️ IMPORTANT: Redémarrez le runtime maintenant
Runtime → Restart runtime


In [11]:
# Configuration quantification 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Charger le tokenizer
print("🔤 Chargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Charger le modèle
print("🧠 Chargement du modèle...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

print("\n✅ Modèle chargé avec succès!")

🔤 Chargement du tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]


✅ Modèle chargé avec succès!


In [12]:
# Configuration LoRA
print("⚙️ Configuration LoRA...")

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# Afficher les paramètres
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"\n📊 Paramètres du modèle:")
print(f"  Total: {total_params:,}")
print(f"  Entraînables: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")

⚙️ Configuration LoRA...

📊 Paramètres du modèle:
  Total: 3,794,014,208
  Entraînables: 41,943,040 (1.11%)


In [13]:
# Préparer les datasets
print("📚 Chargement des datasets...")

train_dataset = load_dataset('json', data_files='data/train_data.jsonl', split='train')
val_dataset = load_dataset('json', data_files='data/val_data.jsonl', split='train')

print(f"  Train: {len(train_dataset)} échantillons")
print(f"  Validation: {len(val_dataset)} échantillons")

# Fonction de formatage
def format_instruction(sample):
    return f"<s>[INST] {sample['instruction']} [/INST] {sample['output']}</s>"

# Fonction de tokenization
def tokenize_function(examples):
    texts = [format_instruction({
        'instruction': inst,
        'output': out
    }) for inst, out in zip(examples['instruction'], examples['output'])]

    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors=None
    )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Tokeniser
print("\n🔄 Tokenization...")
train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

val_dataset = val_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=val_dataset.column_names
)

print("✅ Tokenization terminée!")

📚 Chargement des datasets...


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

  Train: 112 échantillons
  Validation: 14 échantillons

🔄 Tokenization...


Map:   0%|          | 0/112 [00:00<?, ? examples/s]

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

✅ Tokenization terminée!


In [14]:
# Configuration de l'entraînement
print("🏋️ Configuration de l'entraînement...\n")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    warmup_steps=100,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    fp16=True,
    optim="paged_adamw_8bit",
    logging_dir=f"{OUTPUT_DIR}/logs",
    report_to="none",
    gradient_checkpointing=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
)

print(f"📋 Paramètres:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Effective batch: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.
`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


🏋️ Configuration de l'entraînement...

📋 Paramètres:
  Epochs: 3
  Batch size: 4
  Learning rate: 0.0002
  Effective batch: 16


In [15]:
# Créer le Trainer
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

print("✅ Trainer créé!")

✅ Trainer créé!


In [16]:
# ENTRAÎNEMENT
print("="*60)
print("🚀 DÉBUT DE L'ENTRAÎNEMENT")
print("="*60)
print("\n⏰ Durée estimée: 2-4 heures sur T4")
print("📊 Surveillez les logs ci-dessous\n")

# Lancer l'entraînement
trainer.train()

print("\n" + "="*60)
print("✅ ENTRAÎNEMENT TERMINÉ!")
print("="*60)

🚀 DÉBUT DE L'ENTRAÎNEMENT

⏰ Durée estimée: 2-4 heures sur T4
📊 Surveillez les logs ci-dessous



Step,Training Loss,Validation Loss



✅ ENTRAÎNEMENT TERMINÉ!


In [17]:
# Sauvegarder le modèle
print("💾 Sauvegarde du modèle...")

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Modèle sauvegardé dans {OUTPUT_DIR}/")

# Évaluation finale
print("\n📊 Évaluation finale...")
eval_results = trainer.evaluate()

print("\nRésultats:")
for key, value in eval_results.items():
    print(f"  {key}: {value:.4f}")

💾 Sauvegarde du modèle...
✅ Modèle sauvegardé dans ./mistral-incident-detector/

📊 Évaluation finale...



Résultats:
  eval_loss: 0.8402
  eval_runtime: 8.2076
  eval_samples_per_second: 1.7060
  eval_steps_per_second: 0.4870
  epoch: 3.0000


## 📊 ÉTAPE 4: Évaluation du modèle

In [18]:
# Fonction de prédiction
def predict_incident(instruction, max_new_tokens=150):
    """Prédit la réponse pour une instruction"""
    prompt = f"<s>[INST] {instruction} [/INST]"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            repetition_penalty=1.1,
        )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "[/INST]" in full_response:
        response = full_response.split("[/INST]")[-1].strip()
    else:
        response = full_response

    return response

def is_incident_prediction(response):
    """Détermine si c'est un incident"""
    response_lower = response.lower()

    incident_keywords = [
        'incident', 'alerte', 'problème', 'erreur', 'anomalie',
        '🚨', '⚠️', 'down', 'critique', 'panne'
    ]

    normal_keywords = [
        'normal', 'opérationnel', 'aucun incident', 'fonctionne',
        '✅', 'ok', 'bon état', 'nominal'
    ]

    incident_score = sum(1 for kw in incident_keywords if kw in response_lower)
    normal_score = sum(1 for kw in normal_keywords if kw in response_lower)

    return incident_score > normal_score

print("✅ Fonctions de prédiction prêtes")

✅ Fonctions de prédiction prêtes


In [ ]:
# Test sur quelques exemples
print("="*60)
print("🧪 TESTS DU MODÈLE")
print("="*60)

test_cases = [
    "Analyser le statut du service MySQL: status=0, message=, ping=ms",
    "Le service WebSite fonctionne-t-il correctement? Status: 1, Response: 200 - OK",
    "Vérifier l'état de Backend API. Status=1, message=500 - Error, Latency=120ms",
    "Service Sentry: status=1, 200 - OK, ping=350ms",
]

for i, test in enumerate(test_cases, 1):
    print(f"\n{'─'*60}")
    print(f"Test {i}")
    print(f"{'─'*60}")
    print(f"\n📝 Question:\n   {test}")

    response = predict_incident(test)
    is_incident = is_incident_prediction(response)

    print(f"\n🤖 Réponse:\n   {response}")
    print(f"\n🏷️ Classification: {'🚨 INCIDENT' if is_incident else '✅ NORMAL'}")

In [19]:
# Évaluation sur le test set
print("="*60)
print("📈 ÉVALUATION SUR LE TEST SET")
print("="*60)

with open('data/test_data.jsonl', 'r') as f:
    test_data = [json.loads(line) for line in f]

# Limiter à 100 pour la vitesse (retirez [:100] pour tout tester)
test_sample = test_data[:100]

predictions = []
ground_truth = []

print(f"\n🔄 Évaluation de {len(test_sample)} échantillons...\n")

for idx, sample in enumerate(test_sample):
    instruction = sample['instruction']
    true_label = sample['is_incident']

    response = predict_incident(instruction)
    pred_label = is_incident_prediction(response)

    predictions.append(pred_label)
    ground_truth.append(true_label)

    if (idx + 1) % 20 == 0:
        print(f"  Progression: {idx + 1}/{len(test_sample)}")

print("\n✅ Évaluation terminée!")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📈 ÉVALUATION SUR LE TEST SET

🔄 Évaluation de 14 échantillons...



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



✅ Évaluation terminée!


In [20]:
# Calculer les métriques
accuracy = accuracy_score(ground_truth, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(
    ground_truth, predictions, average='binary', zero_division=0
)

cm = confusion_matrix(ground_truth, predictions)
tn, fp, fn, tp = cm.ravel()

print("="*60)
print("📊 RÉSULTATS FINAUX")
print("="*60)

print(f"\n📈 Métriques:")
print(f"  Accuracy:  {accuracy:.3f} ({accuracy*100:.1f}%)")
print(f"  Precision: {precision:.3f}")
print(f"  Recall:    {recall:.3f}")
print(f"  F1-Score:  {f1:.3f}")

print(f"\n🎯 Matrice de confusion:")
print(f"                 Prédit Normal  Prédit Incident")
print(f"  Vrai Normal         {tn:4d}          {fp:4d}")
print(f"  Vrai Incident       {fn:4d}          {tp:4d}")

print(f"\n📝 Interprétation:")
print(f"  ✅ Vrais positifs: {tp} - Incidents correctement détectés")
print(f"  ✅ Vrais négatifs: {tn} - Normaux correctement identifiés")
print(f"  ❌ Faux positifs: {fp} - Fausses alertes")
print(f"  ❌ Faux négatifs: {fn} - Incidents manqués")

if accuracy >= 0.90:
    print("\n🎉 EXCELLENT! Le modèle est prêt pour la production!")
elif accuracy >= 0.80:
    print("\n✅ BON! Performances acceptables, quelques améliorations possibles.")
else:
    print("\n⚠️ Performances insuffisantes. Entraînez plus longtemps ou ajoutez des données.")

📊 RÉSULTATS FINAUX

📈 Métriques:
  Accuracy:  0.929 (92.9%)
  Precision: 1.000
  Recall:    0.857
  F1-Score:  0.923

🎯 Matrice de confusion:
                 Prédit Normal  Prédit Incident
  Vrai Normal            7             0
  Vrai Incident          1             6

📝 Interprétation:
  ✅ Vrais positifs: 6 - Incidents correctement détectés
  ✅ Vrais négatifs: 7 - Normaux correctement identifiés
  ❌ Faux positifs: 0 - Fausses alertes
  ❌ Faux négatifs: 1 - Incidents manqués

🎉 EXCELLENT! Le modèle est prêt pour la production!


## 💾 ÉTAPE 5: Télécharger le modèle

Téléchargez votre modèle fine-tuné pour l'utiliser plus tard.

In [21]:
# Compresser le modèle
print("📦 Compression du modèle...")
!zip -r mistral-incident-detector.zip mistral-incident-detector/
print("✅ Compression terminée!")

📦 Compression du modèle...
  adding: mistral-incident-detector/ (stored 0%)
  adding: mistral-incident-detector/adapter_config.json (deflated 58%)
  adding: mistral-incident-detector/README.md (deflated 66%)
  adding: mistral-incident-detector/chat_template.jinja (deflated 64%)
  adding: mistral-incident-detector/checkpoint-21/ (stored 0%)
  adding: mistral-incident-detector/checkpoint-21/adapter_config.json (deflated 58%)
  adding: mistral-incident-detector/checkpoint-21/README.md (deflated 66%)
  adding: mistral-incident-detector/checkpoint-21/chat_template.jinja (deflated 64%)
  adding: mistral-incident-detector/checkpoint-21/trainer_state.json (deflated 56%)
  adding: mistral-incident-detector/checkpoint-21/scheduler.pt (deflated 56%)
  adding: mistral-incident-detector/checkpoint-21/tokenizer_config.json (deflated 48%)
  adding: mistral-incident-detector/checkpoint-21/adapter_model.safetensors (deflated 8%)
  adding: mistral-incident-detector/checkpoint-21/training_args.bin (defla

In [22]:
# Télécharger
from google.colab import files

print("⬇️ Téléchargement du modèle...")
files.download('mistral-incident-detector.zip')
print("\n✅ Téléchargement lancé! Vérifiez vos téléchargements.")

⬇️ Téléchargement du modèle...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Téléchargement lancé! Vérifiez vos téléchargements.


## 🎉 FÉLICITATIONS!

Vous avez terminé le fine-tuning de Mistral pour la détection d'incidents!

### Prochaines étapes:
1. ✅ Téléchargez votre modèle (voir cellule ci-dessus)
2. ✅ Testez avec vos propres données
3. ✅ Déployez en production (API Flask, FastAPI, etc.)

### Pour sauvegarder sur Google Drive:
```python
from google.colab import drive
drive.mount('/content/drive')
!cp -r mistral-incident-detector /content/drive/MyDrive/
```